In [2]:
import numpy as np

In [ ]:
# Define the parameter
v0 = -50 #mV
eta = 0 #mV
tau_m = 10 #ms
v_rest = 15 #mV
R = 50 #Ohm
beta = 10e5 #s^-1

1. Consider N = 100 non-interacting neurons, with initial voltages $v_i(t = 0)$ uniformly distributed at random between $v_0$ and $η$ (i.e. $v_i(0) ∼ U([v_0,η])$), and submitted to an oscillating external input $I_{ext}(t) = I_0(1 + sin(ωt))$, with $I_0 = 0.5 mA$, $ω = 10^2 s^{−1}(= 0.1 ms^{−1})$. <br>
• Compute the membrane potentials $vi(t)$ and firing times of the neurons, for $t ≤ Tf = 100 ms$. <br>
• Plot the voltage $v_i(t)$ of the first neuron, along with the external input $RI_{ext}(t)$.

In [ ]:
N = 100
vi_0 = np.random.uniform(v0, eta, N)
